<a href="https://colab.research.google.com/github/DA04/machine_learning_course/blob/main/MovieLens_Surprise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Пакет SURPRISE:

*  используйте данные MovieLens 1M
*  можно использовать любые модели из пакета
*  получите RMSE на тестовом сете 0,87 и ниже

Комментарий преподавателя:
В домашнем задании на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе Cross-validation (5 фолдов), а не на отложенном датасете

Данные берете из таблицы ratings.csv (там рейтинг выставлен конкретным пользователем на конкретный фильм) и используете разные алгоритмы из пакета surprise, нужно получить метрику RMSE на кросс-валидации ниже, чем 87

In [2]:
!wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip

--2023-05-04 12:24:56--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K   941KB/s    in 1.0s    

2023-05-04 12:24:59 (941 KB/s) - ‘ml-latest-small.zip’ saved [978202/978202]



In [3]:
!unzip ml-latest-small.zip

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [4]:
import pandas as pd
import numpy as np

In [17]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')

In [18]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [8]:
!pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095416 sha256=223dbc1b45d9ebea03fbc85d003b13f4dd4b3418f7446fd9dad8a1263925c6de
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [43]:
from surprise import KNNWithMeans, KNNBasic
from surprise import NormalPredictor, BaselineOnly
from surprise import SVD, NMF, SVDpp
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate

In [19]:
ratings.rating.min()

0.5

In [20]:
ratings.rating.max()

5.0

In [21]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], Reader(rating_scale=(0.5, 5.0)))

In [23]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

**KNNWithMeans**

In [24]:
algo = KNNWithMeans(k=50, sim_options={
    'name': 'cosine',
    'user_based': True  # compute  similarities between users
})
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [25]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9075


0.9075221620574209

In [27]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9012  0.8926  0.9010  0.9063  0.9000  0.9002  0.0044  
MAE (testset)     0.6895  0.6834  0.6886  0.6932  0.6875  0.6884  0.0032  
Fit time          0.19    0.22    0.20    0.20    0.31    0.23    0.04    
Test time         1.47    1.49    1.59    2.26    1.69    1.70    0.29    


{'test_rmse': array([0.90120509, 0.89259414, 0.90101556, 0.90633204, 0.89999792]),
 'test_mae': array([0.68954271, 0.68335805, 0.68858516, 0.69316847, 0.687495  ]),
 'fit_time': (0.18842720985412598,
  0.21733307838439941,
  0.2046806812286377,
  0.20426559448242188,
  0.31284594535827637),
 'test_time': (1.4678981304168701,
  1.4892799854278564,
  1.5934157371520996,
  2.258283853530884,
  1.689312219619751)}

**KNNBasic**

In [28]:
algo = KNNBasic(k=40, sim_options={
    'name': 'cosine',
    'user_based': True  # compute  similarities between users
})
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [29]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9823


0.9822558142846856

In [30]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9712  0.9657  0.9722  0.9756  0.9737  0.9717  0.0033  
MAE (testset)     0.7463  0.7455  0.7515  0.7499  0.7492  0.7485  0.0022  
Fit time          0.15    0.18    0.27    0.20    0.19    0.20    0.04    
Test time         1.52    2.12    1.63    1.35    1.49    1.62    0.26    


{'test_rmse': array([0.97119638, 0.96571851, 0.97219282, 0.97563831, 0.97373471]),
 'test_mae': array([0.74626263, 0.7455444 , 0.75152561, 0.74986592, 0.74918979]),
 'fit_time': (0.1507575511932373,
  0.1780250072479248,
  0.2655947208404541,
  0.2029128074645996,
  0.18600964546203613),
 'test_time': (1.522313117980957,
  2.119673728942871,
  1.6251726150512695,
  1.352959394454956,
  1.489854097366333)}

**NormalPredictor**

In [31]:
algo = NormalPredictor()
algo.fit(trainset)

In [32]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 1.4304


1.4303502028779382

In [33]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.4270  1.4280  1.4158  1.4115  1.4264  1.4217  0.0068  
MAE (testset)     1.1375  1.1403  1.1290  1.1293  1.1384  1.1349  0.0048  
Fit time          0.29    0.14    0.14    0.14    0.15    0.17    0.06    
Test time         0.33    0.12    0.28    0.11    0.11    0.19    0.09    


{'test_rmse': array([1.42699202, 1.42804869, 1.41578861, 1.41148662, 1.42642424]),
 'test_mae': array([1.13754485, 1.1403226 , 1.12898629, 1.12931883, 1.13838344]),
 'fit_time': (0.29073381423950195,
  0.14270639419555664,
  0.14129304885864258,
  0.13891386985778809,
  0.1467750072479248),
 'test_time': (0.3284749984741211,
  0.12328791618347168,
  0.2842137813568115,
  0.11410951614379883,
  0.11187553405761719)}

**BaselineOnly**

In [34]:
algo = BaselineOnly()
algo.fit(trainset)

Estimating biases using als...


In [35]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8785


0.8785099023109141

In [36]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8764  0.8775  0.8725  0.8692  0.8672  0.8726  0.0040  
MAE (testset)     0.6742  0.6752  0.6746  0.6699  0.6690  0.6726  0.0026  
Fit time          0.37    0.39    0.41    0.40    0.41    0.39    0.01    
Test time         0.07    0.07    0.07    0.07    0.07    0.07    0.00    


{'test_rmse': array([0.87636104, 0.87754034, 0.87254404, 0.86921415, 0.86715334]),
 'test_mae': array([0.67415585, 0.6751697 , 0.67455753, 0.66989364, 0.66904033]),
 'fit_time': (0.37146759033203125,
  0.39145851135253906,
  0.4057962894439697,
  0.3974778652191162,
  0.4075043201446533),
 'test_time': (0.07396507263183594,
  0.07474923133850098,
  0.07210350036621094,
  0.07297563552856445,
  0.07330703735351562)}

**SVD**

In [37]:
algo = SVD()
algo.fit(trainset)

In [38]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8785


0.8784804874598046

In [39]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8801  0.8749  0.8794  0.8711  0.8687  0.8748  0.0045  
MAE (testset)     0.6767  0.6709  0.6751  0.6695  0.6667  0.6718  0.0036  
Fit time          1.36    1.38    1.42    1.39    1.39    1.39    0.02    
Test time         0.13    0.31    0.15    0.31    0.13    0.21    0.09    


{'test_rmse': array([0.88007995, 0.87488965, 0.87941364, 0.87107233, 0.86867525]),
 'test_mae': array([0.67667608, 0.67090392, 0.67506601, 0.66949168, 0.66674397]),
 'fit_time': (1.3576245307922363,
  1.382486343383789,
  1.4171504974365234,
  1.394948959350586,
  1.392585277557373),
 'test_time': (0.13040518760681152,
  0.311002254486084,
  0.14711952209472656,
  0.30906224250793457,
  0.13286209106445312)}

**SVD++**

In [46]:
algo = SVDpp()
algo.fit(trainset)

In [47]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8643


0.8642904871062059

In [48]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8669  0.8675  0.8669  0.8441  0.8611  0.8613  0.0089  
MAE (testset)     0.6626  0.6674  0.6624  0.6489  0.6558  0.6594  0.0065  
Fit time          97.77   97.45   97.34   96.70   96.13   97.08   0.59    
Test time         11.33   10.85   11.71   11.65   11.25   11.36   0.31    


{'test_rmse': array([0.86687854, 0.8675322 , 0.86691532, 0.84409062, 0.86113191]),
 'test_mae': array([0.66255106, 0.66743599, 0.6624243 , 0.64885764, 0.65575212]),
 'fit_time': (97.77122783660889,
  97.4493191242218,
  97.33537006378174,
  96.69618654251099,
  96.12556433677673),
 'test_time': (11.32949948310852,
  10.852134227752686,
  11.711177349090576,
  11.651665925979614,
  11.254899740219116)}

**NMF**

In [40]:
algo = NMF()
algo.fit(trainset)

In [41]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9335


0.9334976652047904

In [42]:
cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9289  0.9202  0.9144  0.9282  0.9145  0.9212  0.0063  
MAE (testset)     0.7098  0.7041  0.7033  0.7106  0.6991  0.7054  0.0043  
Fit time          2.57    2.77    3.05    2.60    2.71    2.74    0.17    
Test time         0.15    0.20    0.21    0.11    0.38    0.21    0.09    


{'test_rmse': array([0.92894425, 0.92024796, 0.91439096, 0.92817427, 0.91445591]),
 'test_mae': array([0.70977888, 0.70413493, 0.70326294, 0.71057648, 0.69910169]),
 'fit_time': (2.5676941871643066,
  2.771426200866699,
  3.048194646835327,
  2.6013879776000977,
  2.7068004608154297),
 'test_time': (0.15151596069335938,
  0.2012033462524414,
  0.21045160293579102,
  0.11104059219360352,
  0.37751197814941406)}